In [1]:
%load_ext autoreload
%autoreload 2

import torch as t
import numpy as np

from memory_module import TGNPLMemory
from msg_func import TGNPLMessage
from msg_agg import *

# Test memory

In [2]:
num_nodes = 10
num_prods = 2
raw_msg_dim = 1
state_dim = 10
time_dim = 1
message_module = TGNPLMessage(raw_msg_dim, state_dim, time_dim)
aggregator_module = MeanAggregator()

In [13]:
# test initialization
mem = TGNPLMemory(num_nodes,
        num_prods,
        raw_msg_dim,
        state_dim,
        time_dim,
        message_module,
        aggregator_module,
        state_updater_cell="gru",
        use_inventory=False,
        debug=True)

In [14]:
# before any interactions have been added
np.random.seed(0)
n_id = np.random.choice(num_nodes, size=5, replace=False)
n_id = t.from_numpy(n_id)
print(n_id)
memory, last_update = mem(n_id)  # test .forward()

tensor([2, 8, 4, 9, 1])
msg_s tensor([], size=(0, 32), grad_fn=<CatBackward0>)
msg_d tensor([], size=(0, 32), grad_fn=<CatBackward0>)
msg_p tensor([], size=(0, 32), grad_fn=<CatBackward0>)
aggr tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]], grad_fn=<DivBackward0>)
memory tensor([[ 0.0011,  0.1683,  0.0205, -0.

In [15]:
memory

tensor([[ 0.0011,  0.1683,  0.0205, -0.1634, -0.0667,  0.0969,  0.0471, -0.0684,
          0.0764, -0.0488],
        [ 0.0011,  0.1683,  0.0205, -0.1634, -0.0667,  0.0969,  0.0471, -0.0684,
          0.0764, -0.0488],
        [ 0.0011,  0.1683,  0.0205, -0.1634, -0.0667,  0.0969,  0.0471, -0.0684,
          0.0764, -0.0488],
        [ 0.0011,  0.1683,  0.0205, -0.1634, -0.0667,  0.0969,  0.0471, -0.0684,
          0.0764, -0.0488],
        [ 0.0011,  0.1683,  0.0205, -0.1634, -0.0667,  0.0969,  0.0471, -0.0684,
          0.0764, -0.0488]], grad_fn=<CatBackward0>)

In [16]:
last_update

tensor([0, 0, 0, 0, 0])

In [17]:
# try adding interactions
# expectation: 
# 1) get_updated_memory will print 6 nodes, but memories shouldn't be updated yet
# 2) _update_msg_store should update all three msg stores
src = t.Tensor([0, 0, 1, 2]).long()
dst = t.Tensor([3, 3, 3, 0]).long()
prod = t.Tensor([8, 8, 8, 9]).long()
time = t.Tensor(np.zeros(4))
raw_msg = t.Tensor([10, 20, 5, 13]).reshape(-1, 1)
mem.update_state(src, dst, prod, time, raw_msg)

msg_s tensor([], size=(0, 32), grad_fn=<CatBackward0>)
msg_d tensor([], size=(0, 32), grad_fn=<CatBackward0>)
msg_p tensor([], size=(0, 32), grad_fn=<CatBackward0>)
aggr tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [18]:
mem.msg_s_store

{0: (tensor([0, 0]),
  tensor([3, 3]),
  tensor([8, 8]),
  tensor([0., 0.]),
  tensor([[10.],
          [20.]])),
 1: (tensor([1]), tensor([3]), tensor([8]), tensor([0.]), tensor([[5.]])),
 2: (tensor([2]), tensor([0]), tensor([9]), tensor([0.]), tensor([[13.]])),
 3: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 4: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 5: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 6: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 7: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], d

In [19]:
mem.msg_d_store

{0: (tensor([2]), tensor([0]), tensor([9]), tensor([0.]), tensor([[13.]])),
 1: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 2: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 3: (tensor([0, 0, 1]),
  tensor([3, 3, 3]),
  tensor([8, 8, 8]),
  tensor([0., 0., 0.]),
  tensor([[10.],
          [20.],
          [ 5.]])),
 4: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 5: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 6: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.i

In [20]:
mem.msg_p_store

{0: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 1: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 2: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 3: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 4: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 5: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 6: (tensor([], dtype=

In [21]:
# now get memory for all nodes - only nodes with interactions should've changed
n_id = t.from_numpy(np.arange(num_nodes))
memory, last_update = mem(n_id)  # test .forward()

msg_s tensor([[ 1.0538e-03,  1.6834e-01,  2.0486e-02, -1.6342e-01, -6.6716e-02,
          9.6874e-02,  4.7055e-02, -6.8389e-02,  7.6362e-02, -4.8766e-02,
          1.0538e-03,  1.6834e-01,  2.0486e-02, -1.6342e-01, -6.6716e-02,
          9.6874e-02,  4.7055e-02, -6.8389e-02,  7.6362e-02, -4.8766e-02,
          1.0538e-03,  1.6834e-01,  2.0486e-02, -1.6342e-01, -6.6716e-02,
          9.6874e-02,  4.7055e-02, -6.8389e-02,  7.6362e-02, -4.8766e-02,
          1.0000e+01,  7.8378e-01],
        [ 1.0538e-03,  1.6834e-01,  2.0486e-02, -1.6342e-01, -6.6716e-02,
          9.6874e-02,  4.7055e-02, -6.8389e-02,  7.6362e-02, -4.8766e-02,
          1.0538e-03,  1.6834e-01,  2.0486e-02, -1.6342e-01, -6.6716e-02,
          9.6874e-02,  4.7055e-02, -6.8389e-02,  7.6362e-02, -4.8766e-02,
          1.0538e-03,  1.6834e-01,  2.0486e-02, -1.6342e-01, -6.6716e-02,
          9.6874e-02,  4.7055e-02, -6.8389e-02,  7.6362e-02, -4.8766e-02,
          2.0000e+01,  7.8378e-01],
        [ 1.0538e-03,  1.6834e-01,

In [25]:
# 2 and 9 same
# 3 and 8 same
# makes sense - they are involved in the same transactions and we are using
# identity for message module
affected = t.cat([src, dst, prod]).unique()
print(affected)
memory[affected]

tensor([0, 1, 2, 3, 8, 9])


tensor([[ 8.9771e-01,  6.3795e-01,  3.8965e-01,  2.4443e-01, -3.6765e-02,
          2.7950e-01, -9.6395e-01,  6.9615e-01,  9.3348e-01, -4.2066e-02],
        [ 3.7223e-01,  6.0533e-01,  4.4263e-01, -1.7851e-01,  8.9393e-02,
          2.9187e-01, -5.9421e-01,  3.6137e-01,  5.1510e-01, -6.4611e-04],
        [ 8.6636e-01,  6.3673e-01,  4.0251e-01,  1.7883e-01, -2.5082e-02,
          2.9071e-01, -9.4978e-01,  6.7092e-01,  9.0905e-01, -3.9330e-02],
        [ 8.2515e-01,  6.3535e-01,  4.1521e-01,  1.1274e-01, -9.6916e-03,
          3.0028e-01, -9.2981e-01,  6.4207e-01,  8.7631e-01, -3.5641e-02],
        [ 8.2515e-01,  6.3535e-01,  4.1521e-01,  1.1274e-01, -9.6916e-03,
          3.0028e-01, -9.2981e-01,  6.4207e-01,  8.7631e-01, -3.5641e-02],
        [ 8.6636e-01,  6.3673e-01,  4.0251e-01,  1.7883e-01, -2.5082e-02,
          2.9071e-01, -9.4978e-01,  6.7092e-01,  9.0905e-01, -3.9330e-02]],
       grad_fn=<IndexBackward0>)

In [24]:
# all the same
unaffected = t.Tensor([4, 5, 6, 7]).long()
memory[unaffected]

tensor([[ 0.0011,  0.1683,  0.0205, -0.1634, -0.0667,  0.0969,  0.0471, -0.0684,
          0.0764, -0.0488],
        [ 0.0011,  0.1683,  0.0205, -0.1634, -0.0667,  0.0969,  0.0471, -0.0684,
          0.0764, -0.0488],
        [ 0.0011,  0.1683,  0.0205, -0.1634, -0.0667,  0.0969,  0.0471, -0.0684,
          0.0764, -0.0488],
        [ 0.0011,  0.1683,  0.0205, -0.1634, -0.0667,  0.0969,  0.0471, -0.0684,
          0.0764, -0.0488]], grad_fn=<IndexBackward0>)